In [ ]:
import torch
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
from pymilvus import MilvusClient, DataType

In [ ]:
# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

# 2. Create a collection in quick setup mode
client.create_collection(
    collection_name="quick_setup",
    dimension=5
)

In the above setup,
- The primary and vector fields use their default names (<b>id</b> and <b>vector</b>)
- The metric type is also set to its default value (<b>COSINE</b>)
- The primary field accepts integers and does not automatically increments
- A reserved JSON field named <b>$meta</b> is used to store non-schema-defiend fields and their values

In [7]:
# 4. Insert data into the collection
# 4.1. Prepare data
data=[
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
    {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298"},
    {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794"},
    {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222"},
    {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392"},
    {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510"},
    {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381"},
    {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976"}
]

In [8]:
# 4.2. Insert data
res = client.insert(
    collection_name="quick_setup",
    data=data
)


In [9]:
print(res)

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


In [10]:
# 6. Search with a single vector
# 6.1. Prepare query vectors
query_vectors = [
    [0.041732933, 0.013779674, -0.027564144, -0.013061441, 0.009748648]
]

In [17]:
# 6.2. Start search
res = client.search(
    collection_name="quick_setup",     # target collection
    data=query_vectors,                # query vectors
    filter='$meta["color"] like "red%"',      # filter
    limit=3,                           # number of returned entities
)

In [18]:
print(res)

[[{'id': 6, 'distance': 0.5555188655853271, 'entity': {}}, {'id': 1, 'distance': -0.08607686311006546, 'entity': {}}, {'id': 4, 'distance': -0.2536149024963379, 'entity': {}}]]


In [ ]:
# 15. Drop collection
client.drop_collection(
    collection_name="quick_setup"
)

# Collections
Collection은 전통적인 RDB(MySQL, MSSQL, ...)에서의 테이블과 비슷한 개념

In [29]:
# 6. List all collection names
res = client.list_collections()
print(res)

['quick_setup']


In [30]:
res = client.list_indexes(
    collection_name="quick_setup"
)
print(res)

['vector']


In [25]:
# 7. Load the collection
client.load_collection(
    collection_name="quick_setup",
    replica_number=1 # Number of replicas to create on query nodes. Max value is 1 for Milvus Standalone, and no greater than `queryNode.replicas` for Milvus Cluster.
)

In [26]:
res = client.get_load_state(
    collection_name="quick_setup"
)
print(res)

{'state': <LoadState: Loaded>}


# Database

In [19]:
from pymilvus import connections, db

In [21]:
conn = connections.connect(host="127.0.0.1", port=19530)

In [22]:
db.list_database()

['default']

In [23]:
db.using_database('default')